[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ma7555/kerasgen/blob/master/examples/triplet_loss.ipynb)

In [12]:
%git clone https://github.com/ma7555/kerasgen
%cd kerasgen
%pip install .
%pip install tensorflow-addons

c:\Users\ma7mo\Documents\GitHub\kerasgen\examples\kerasgen


fatal: destination path 'kerasgen' already exists and is not an empty directory.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


Processing c:\users\ma7mo\documents\github\kerasgen\examples\kerasgen
  Created wheel for kerasgen: filename=kerasgen-0.1.2-py3-none-any.whl size=8754 sha256=e42ab5abad6c585436908e7f1c945478ed6630ec5956177f3f4852396c886951
  Stored in directory: C:\Users\ma7mo\AppData\Local\Temp\pip-ephem-wheel-cache-e761ru5_\wheels\cb\c6\cd\2ea5f4585a7b6a76b39fed4d4bd1566142fab984d4d2071d92
Successfully built kerasgen


In [3]:
from kerasgen.balanced_image_dataset import balanced_image_dataset_from_directory
import tensorflow as tf
import tensorflow_addons as tfa

In [8]:
directory = './sample'

train_ds = balanced_image_dataset_from_directory(
    directory, num_classes_per_batch=2,
    num_images_per_class=4, image_size=(256, 256),
    validation_split=0.2, subset='training', seed=555,
    safe_triplet=True)

val_ds = balanced_image_dataset_from_directory(
    directory, num_classes_per_batch=2,
    num_images_per_class=4, image_size=(256, 256),
    validation_split=0.2, subset='validation', seed=555,
    safe_triplet=True)

Found 75 files belonging to 5 classes.
Using 60 files for training.
Found 75 files belonging to 5 classes.
Using 15 files for validation.


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

In [10]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [11]:
# Train the network
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
8/8 [==============================] - 8s 758ms/step - loss: 0.8873 - val_loss: 0.9057
Epoch 2/5
8/8 [==============================] - 5s 675ms/step - loss: 0.6348 - val_loss: 0.9125
Epoch 3/5
8/8 [==============================] - 5s 595ms/step - loss: 0.4327 - val_loss: 0.7794
Epoch 4/5
8/8 [==============================] - 5s 606ms/step - loss: 0.5359 - val_loss: 0.8979
Epoch 5/5
8/8 [==============================] - 5s 600ms/step - loss: 0.3328 - val_loss: 0.8185
